# DEAM Dataset - Preprocessing

# Done on local computer

## Import relevant libraries

In [ ]:
import numpy as np
import ast
import pandas as pd
import matplotlib.pyplot as plt

import os

Set file directory variables

In [ ]:
import sys
sys.path.insert(1, '../../utils')
from paths import *

## Verify file count

Check that the number of .csv files in the './data/DEAM/features' directory matches the number number of .mp3 files in the './data/DEAM/MEMD_audio' directory

In [ ]:
def extract_filenames(directory, file_format):
    raw_file_list = os.listdir(directory)
    raw_count = len(raw_file_list)
    print(f"Total files: {raw_count} \nFiles: {raw_file_list}\n")
    print("Processing files...")
    processed_list = []
    for filename in raw_file_list:
        if file_format in filename:
            filename = filename.replace(f".{file_format}", "")
            processed_list.append(filename)
    
    processed_count = len(processed_list)
    print(f"Total files: {processed_count} \nFiles: {processed_list}")
    return processed_list, processed_count

In [ ]:
features_list, features_count = extract_filenames(get_deam_path('features'), "csv")

In [ ]:
audio_list, audio_count = extract_filenames(get_deam_path('MEMD_audio'), "mp3")

In [ ]:
if set(features_list) == set(audio_list):
    print("Features and Audio Files match!")
else:
    diff = set(features_list) - set(audio_list)
    print("Elements absent in audio:", diff)

## Process the annotation dataframes

Read static song-level annotations with song_id from 1 to 2000 (2013 and 2014 only)

In [ ]:
df_annotations = pd.read_csv(get_deam_path('annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv'))
display(df_annotations)
print(df_annotations.shape)

Map the valence and arousal values in the dataset, ranging from 1 to 9, to values ranging from -1 to 1, to follow convention

In [ ]:
def map_va_value(value):
  old_min = 1
  old_max = 9

  new_min = -1
  new_max = 1

  mapped_value = ((value - old_min) * (new_max - new_min) / (old_max - old_min)) + new_min
  return mapped_value

# Test the function
for i in range(1, 10):
    print(f"Original value: {i}, Mapped value: {map_va_value(i)}")

In [ ]:
df_annotations['valence_mean_mapped'] = df_annotations[' valence_mean'].apply(map_va_value)
df_annotations['arousal_mean_mapped'] = df_annotations[' arousal_mean'].apply(map_va_value)
df_annotations = df_annotations.drop([' valence_mean', ' valence_std', ' arousal_mean', ' arousal_std'], axis=1)

In [ ]:
df_annotations

Export the dataframe

In [ ]:
df_annotations.to_csv(get_deam_path('processed/annotations/deam_static_annotations.csv'), index=False)

Import the dataframe

In [ ]:
df_annotations = pd.read_csv(get_deam_path('processed/annotations/deam_static_annotations.csv'))
df_annotations

## Process the Essentia features datasets

Define function to check for any non-float/int columns and deal with them

In [ ]:
df_essentia_features = pd.read_csv(get_deam_path('processed/features/essentia_features.csv'))
df_essentia_features

Drop the 'Unnamed: 0	' column

In [ ]:
# drop Unnamed:0 column
df_essentia_features = df_essentia_features[df_essentia_features.columns[1:]]
df_essentia_features

See what features are available

In [ ]:
print(df_essentia_features.columns.to_list())

Get song_ids

In [ ]:
song_ids = df_essentia_features['song_id'].values.tolist()
print(song_ids)

Some features are irrelevant, or are metadata. Drop them

In [ ]:
metadata_columns = [col for col in df_essentia_features.columns if 'metadata' in col]
df_essentia_features = df_essentia_features.drop(columns=metadata_columns)
df_essentia_features

Find out if any columns are not of the type float or int

In [ ]:
pd.set_option('display.max_columns', None)
df_essentia_features.select_dtypes(exclude=['int64', 'float64'])

Seems like there are 3 main types of column types:

1. ndarray
2. string
3. float64 or int64

Get the columns whose type is ndarray

In [ ]:
pd.reset_option('display.max_columns')

string_columns = ['tonal.chords_key',
                  'tonal.chords_scale',
                  'tonal.key_edma.key',
                  'tonal.key_edma.scale',
                  'tonal.key_krumhansl.key',
                  'tonal.key_krumhansl.scale',
                  'tonal.key_temperley.key',
                  'tonal.key_temperley.scale'
                  ]

df_essentia_features_ndarray_columns = df_essentia_features.select_dtypes(exclude=['int64', 'float64'])
ndarray_columns = df_essentia_features_ndarray_columns.columns.difference(string_columns)
df_essentia_features_ndarray_columns = df_essentia_features_ndarray_columns[ndarray_columns]
df_essentia_features_ndarray_columns

Print one random value

In [ ]:
print(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0])

Flatten the columns whose values are ndarrays, like tonal.chords_histogram

Credits to https://stackoverflow.com/questions/45704999/how-to-convert-vector-wrapped-as-string-to-numpy-array-in-pandas-dataframe

In [ ]:
def string_to_ndarray(str):
  return np.fromstring(str.replace('\n','')
                       .replace('[','')
                       .replace(']','')
                       .replace('  ',' '), 
                       sep=' '
                       )

In [ ]:
df_essentia_features_ndarray_columns = df_essentia_features_ndarray_columns.applymap(string_to_ndarray)
df_essentia_features_ndarray_columns

Print out that same value to ensure that it is properly converted to a ndarray

In [ ]:
print(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0])
print(type(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0]))

In [ ]:
pd.reset_option('display.max_columns')

Now that all values are proper ndarrays, let's flatten these columns, define a function to flatten a column into multiple new columns containing float64

In [ ]:
def flatten_column(df, col):
  print(col)
  result_dict = {}
  num_of_new_cols = max([len(i) for i in df[col]])
  # num_of_new_cols = len(df[col][0])
  num_of_rows = len(df[col])

  for i in range(num_of_new_cols):
    result_col_name = f'{col}_{i}'
    result_dict[result_col_name] = []

  for i in range(num_of_rows):
    for j in range(num_of_new_cols):
      result_col_name = f'{col}_{j}'

      # do padding
      if j >= len(df[col][i]):
        value = 0
      else:
        value = df[col][i][j]
      
      result_dict[result_col_name].append(value)

  return pd.DataFrame(result_dict)

# test the function
df = flatten_column(df_essentia_features_ndarray_columns, 'lowlevel.mfcc.icov')
df

Apply the function to flatten all these columns

In [ ]:
ndarray_columns = df_essentia_features_ndarray_columns.columns.to_list()
df_ndarray_columns = []

for column in ndarray_columns:
  df_ndarray_column = flatten_column(df_essentia_features_ndarray_columns, column)
  df_ndarray_columns.append(df_ndarray_column)

df_essentia_features_ndarray_columns = pd.concat(df_ndarray_columns, axis=1)
df_essentia_features_ndarray_columns.insert(0, column='song_id', value=song_ids)
df_essentia_features_ndarray_columns

For the string columns, convert these categorical data into numerical data, get the dataframe with only the string columns first

In [ ]:
df_essentia_features_string_columns = df_essentia_features[string_columns]
df_essentia_features_string_columns

Then use cat.codes attribute to convert these categorical columns into numerical columns

In [ ]:
for col in df_essentia_features_string_columns.columns:
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].astype('category')
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].cat.codes

df_essentia_features_string_columns.insert(0, column='song_id', value=song_ids)
df_essentia_features_string_columns

Combine the ndarray columns, string columns, and the rest of the dataframe together in one flattened dataframe with just numerical data

In [ ]:
df_essentia_features_numerical_columns = df_essentia_features.select_dtypes(include=['int64', 'float64'])

df_temp = pd.merge(df_essentia_features_numerical_columns, df_essentia_features_ndarray_columns, how='inner', on='song_id')
df_essentia_features_flattened = pd.merge(df_temp, df_essentia_features_string_columns, how='inner', on='song_id')

df_essentia_features_flattened

Export the flattened Essentia features dataset

In [ ]:
df_essentia_features_flattened.to_csv(get_deam_path('processed/features/essentia_features_flattened.csv'))

## Create separate, more distinct features dataframes extracted by Essentia

Import the features lsit

In [ ]:
from deam_essentia_best_features import *

### Get song ids for songs in 2013 and 2014

In [ ]:
audio_path = get_deam_path('MEMD_audio')
song_ids_temp = []

# Iterate through all files in the directory
for filename in os.listdir(audio_path):
    # Check if the path is a file (not a subdirectory)
    if os.path.isfile(os.path.join(audio_path, filename)):
        song_ids_temp.append(int(filename[:-4]))

song_ids_temp.sort()
song_ids = []

# remove all song_ids from 2015 (song_id 2001 onwards)
for song_id in song_ids_temp:
    if song_id <= 2000:
        song_ids.append(song_id)

### Create separate feature dataset for best features for building arousal regressor
According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [ ]:
df_essentia_best_arousal_features = df_essentia_features_flattened[deam_essentia_best_arousal_features_flattened]
df_essentia_best_arousal_features.insert(0, 'song_id', song_ids)
df_essentia_best_arousal_features.to_csv(get_deam_path('processed/features/essentia_best_arousal_features.csv'))

Import the best feature dataset .csv for arousal

In [ ]:
df_essentia_best_arousal_features = pd.read_csv(get_deam_path('processed/features/essentia_best_arousal_features.csv'))

# drop Unnamed:0 column
df_essentia_best_arousal_features = df_essentia_best_arousal_features[df_essentia_best_arousal_features.columns[1:]]

df_essentia_best_arousal_features

### Create separate feature dataset for best features for building valence regressor
According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [ ]:
df_essentia_best_valence_features = df_essentia_features_flattened[deam_essentia_best_valence_features_flattened]
df_essentia_best_valence_features.insert(0, 'song_id', song_ids)
df_essentia_best_valence_features.to_csv(get_deam_path('processed/features/essentia_best_valence_features.csv'))

Import the best feature dataset .csv for valence

In [ ]:
df_essentia_best_valence_features = pd.read_csv(get_deam_path('processed/features/essentia_best_valence_features.csv'))

# drop Unnamed:0 column
df_essentia_best_valence_features = df_essentia_best_valence_features[df_essentia_best_valence_features.columns[1:]]

df_essentia_best_valence_features

### Create separate feature dataset for best overall features for detecting both arousal and valence

According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [ ]:
df_essentia_best_overall_features = df_essentia_features_flattened[deam_essentia_best_overall_features_flattened]
df_essentia_best_overall_features.insert(0, 'song_id', song_ids)
df_essentia_best_overall_features.to_csv(get_deam_path('processed/features/essentia_best_overall_features.csv'))

Import the best feature dataset .csv overall 

In [ ]:
df_essentia_best_overall_features = pd.read_csv(get_deam_path('processed/features/essentia_best_overall_features.csv'))

# drop Unnamed:0 column
df_essentia_best_overall_features = df_essentia_best_overall_features[df_essentia_best_overall_features.columns[1:]]

df_essentia_best_overall_features

## Integrate Essentia features into openSMILE features

Import openSMILE featuresets .csv

In [ ]:
df_deam_opensmile_compare2016_features = pd.read_csv(get_deam_path('processed/features/opensmile_compare2016_features.csv'))
df_deam_opensmile_emobase_features = pd.read_csv(get_deam_path('processed/features/opensmile_emobase_features.csv'))
df_deam_opensmile_gemaps_features = pd.read_csv(get_deam_path('processed/features/opensmile_gemaps_features.csv'))
df_deam_opensmile_egemaps_features = pd.read_csv(get_deam_path('processed/features/opensmile_egemaps_features.csv'))

# drop Unnamed:0 column
df_deam_opensmile_compare2016_features = df_deam_opensmile_compare2016_features[df_deam_opensmile_compare2016_features.columns[1:]]
df_deam_opensmile_emobase_features = df_deam_opensmile_emobase_features[df_deam_opensmile_emobase_features.columns[1:]]
df_deam_opensmile_gemaps_features = df_deam_opensmile_gemaps_features[df_deam_opensmile_gemaps_features.columns[1:]]
df_deam_opensmile_egemaps_features = df_deam_opensmile_egemaps_features[df_deam_opensmile_egemaps_features.columns[1:]]

In [ ]:
df_deam_opensmile_compare2016_features

In [ ]:
df_deam_opensmile_emobase_features

In [ ]:
df_deam_opensmile_gemaps_features

In [ ]:
df_deam_opensmile_egemaps_features

Integrate Essentia all features into openSMILE ComParE2016 features

In [ ]:
df_deam_integrated_essentia_all_opensmile_compare2016 = pd.merge(df_essentia_features_flattened, df_deam_opensmile_compare2016_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_all_opensmile_compare2016.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_all_opensmile_compare2016.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_all_opensmile_compare2016

In [ ]:
df_deam_integrated_essentia_all_opensmile_compare2016.to_csv(get_deam_path('processed/features/integrated/essentia_all_opensmile_compare2016_features.csv'))

Integrate Essentia all features into openSMILE emobase features

In [ ]:
df_deam_integrated_essentia_all_opensmile_emobase = pd.merge(df_essentia_features_flattened, df_deam_opensmile_emobase_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_all_opensmile_emobase.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_all_opensmile_emobase.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_all_opensmile_emobase

In [ ]:
df_deam_integrated_essentia_all_opensmile_emobase.to_csv(get_deam_path('processed/features/integrated/essentia_all_opensmile_emobase_features.csv'))

Integrate Essentia all features into openSMILE GeMAPS features

In [ ]:
df_deam_integrated_essentia_all_opensmile_gemaps = pd.merge(df_essentia_features_flattened, df_deam_opensmile_gemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_all_opensmile_gemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_all_opensmile_gemaps.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_all_opensmile_gemaps

In [ ]:
df_deam_integrated_essentia_all_opensmile_gemaps.to_csv(get_deam_path('processed/features/integrated/essentia_all_opensmile_gemaps_features.csv'))

Integrate Essentia all features into openSMILE eGeMAPS features

In [ ]:
df_deam_integrated_essentia_all_opensmile_egemaps = pd.merge(df_essentia_features_flattened, df_deam_opensmile_egemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_all_opensmile_egemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_all_opensmile_egemaps.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_all_opensmile_egemaps

In [ ]:
df_deam_integrated_essentia_all_opensmile_egemaps.to_csv(get_deam_path('processed/features/integrated/essentia_all_opensmile_egemaps_features.csv'))

Integrate Essentia best overall features into openSMILE ComParE2016 features

In [ ]:
df_deam_integrated_essentia_best_overall_opensmile_compare2016 = pd.merge(df_essentia_best_overall_features, df_deam_opensmile_compare2016_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_best_overall_opensmile_compare2016.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_best_overall_opensmile_compare2016.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_best_overall_opensmile_compare2016

In [ ]:
df_deam_integrated_essentia_best_overall_opensmile_compare2016.to_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_compare2016_features.csv'))

Integrate Essentia best overall features into openSMILE emobase features

In [ ]:
df_deam_integrated_essentia_best_overall_opensmile_emobase = pd.merge(df_essentia_best_overall_features, df_deam_opensmile_emobase_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_best_overall_opensmile_emobase.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_best_overall_opensmile_emobase.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_best_overall_opensmile_emobase

In [ ]:
df_deam_integrated_essentia_best_overall_opensmile_emobase.to_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_emobase_features.csv'))

Integrate Essentia best overall features into openSMILE GeMAPS features

In [ ]:
df_deam_integrated_essentia_best_overall_opensmile_gemaps = pd.merge(df_essentia_best_overall_features, df_deam_opensmile_gemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_best_overall_opensmile_gemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_best_overall_opensmile_gemaps.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_best_overall_opensmile_gemaps

In [ ]:
df_deam_integrated_essentia_best_overall_opensmile_gemaps.to_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_gemaps_features.csv'))

Integrate Essentia best overall features into openSMILE eGeMAPS features

In [ ]:
df_deam_integrated_essentia_best_overall_opensmile_egemaps = pd.merge(df_essentia_best_overall_features, df_deam_opensmile_egemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_deam_integrated_essentia_best_overall_opensmile_egemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_deam_integrated_essentia_best_overall_opensmile_egemaps.drop(columns=identical_cols, inplace=True)

df_deam_integrated_essentia_best_overall_opensmile_egemaps

In [ ]:
df_deam_integrated_essentia_best_overall_opensmile_egemaps.to_csv(get_deam_path('processed/features/integrated/essentia_best_overall_opensmile_egemaps_features.csv'))

## Data Scaling

Import relevant libraries

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### Data Standardisation

In [ ]:
scaler = StandardScaler()

Essentia All + openSMILE ComParE2016

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_compare2016_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_compare2016), columns=df_deam_integrated_essentia_all_opensmile_compare2016.columns)

df_deam_integrated_essentia_all_opensmile_compare2016_standardised = df_deam_integrated_essentia_all_opensmile_compare2016_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_compare2016_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_compare2016_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_all_opensmile_compare2016_features.csv'))

df_deam_integrated_essentia_all_opensmile_compare2016_standardised

Essentia All + openSMILE emobase

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_emobase_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_emobase), columns=df_deam_integrated_essentia_all_opensmile_emobase.columns)

df_deam_integrated_essentia_all_opensmile_emobase_standardised = df_deam_integrated_essentia_all_opensmile_emobase_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_emobase_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_emobase_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_all_opensmile_emobase_features.csv'))

df_deam_integrated_essentia_all_opensmile_emobase_standardised

Essentia All + openSMILE GeMAPS

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_gemaps_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_gemaps), columns=df_deam_integrated_essentia_all_opensmile_gemaps.columns)

df_deam_integrated_essentia_all_opensmile_gemaps_standardised = df_deam_integrated_essentia_all_opensmile_gemaps_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_gemaps_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_gemaps_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_all_opensmile_gemaps_features.csv'))

df_deam_integrated_essentia_all_opensmile_gemaps_standardised

Essentia All + openSMILE eGeMAPS

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_egemaps_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_egemaps), columns=df_deam_integrated_essentia_all_opensmile_egemaps.columns)

df_deam_integrated_essentia_all_opensmile_egemaps_standardised = df_deam_integrated_essentia_all_opensmile_egemaps_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_egemaps_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_egemaps_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_all_opensmile_egemaps_features.csv'))

df_deam_integrated_essentia_all_opensmile_egemaps_standardised

Essentia Best Overall + openSMILE ComParE2016

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_compare2016), columns=df_deam_integrated_essentia_best_overall_opensmile_compare2016.columns)

df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised = df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_compare2016_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_compare2016_standardised

Essentia Best Overall + openSMILE emobase

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_emobase), columns=df_deam_integrated_essentia_best_overall_opensmile_emobase.columns)

df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised = df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_emobase_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_emobase_standardised

Essentia Best Overall + openSMILE GeMAPS

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_gemaps), columns=df_deam_integrated_essentia_best_overall_opensmile_gemaps.columns)

df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised = df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_gemaps_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_gemaps_standardised

Essentia Best Overall + openSMILE eGeMAPS

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_egemaps), columns=df_deam_integrated_essentia_best_overall_opensmile_egemaps.columns)

df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised = df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised.to_csv(get_deam_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_egemaps_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_egemaps_standardised

### Data Normalisation

In [ ]:
scaler = MinMaxScaler()

Essentia All + openSMILE ComParE2016

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_compare2016_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_compare2016), columns=df_deam_integrated_essentia_all_opensmile_compare2016.columns)

df_deam_integrated_essentia_all_opensmile_compare2016_normalised = df_deam_integrated_essentia_all_opensmile_compare2016_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_compare2016_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_compare2016_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_all_opensmile_compare2016_features.csv'))

df_deam_integrated_essentia_all_opensmile_compare2016_normalised

Essentia All + openSMILE emobase

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_emobase_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_emobase), columns=df_deam_integrated_essentia_all_opensmile_emobase.columns)

df_deam_integrated_essentia_all_opensmile_emobase_normalised = df_deam_integrated_essentia_all_opensmile_emobase_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_emobase_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_emobase_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_all_opensmile_emobase_features.csv'))

df_deam_integrated_essentia_all_opensmile_emobase_normalised

Essentia All + openSMILE GeMAPs

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_gemaps_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_gemaps), columns=df_deam_integrated_essentia_all_opensmile_gemaps.columns)

df_deam_integrated_essentia_all_opensmile_gemaps_normalised = df_deam_integrated_essentia_all_opensmile_gemaps_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_gemaps_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_gemaps_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_all_opensmile_gemaps_features.csv'))

df_deam_integrated_essentia_all_opensmile_gemaps_normalised

Essentia All + openSMILE eGeMAPS

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_all_opensmile_egemaps_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_all_opensmile_egemaps), columns=df_deam_integrated_essentia_all_opensmile_egemaps.columns)

df_deam_integrated_essentia_all_opensmile_egemaps_normalised = df_deam_integrated_essentia_all_opensmile_egemaps_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_all_opensmile_egemaps_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_all_opensmile_egemaps_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_all_opensmile_egemaps_features.csv'))

df_deam_integrated_essentia_all_opensmile_egemaps_normalised

Essentia Best Overall + openSMILE ComParE2016

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_compare2016), columns=df_deam_integrated_essentia_best_overall_opensmile_compare2016.columns)

df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised = df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_compare2016_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_compare2016_normalised

Essentia Best Overall + openSMILE emobase

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_emobase), columns=df_deam_integrated_essentia_best_overall_opensmile_emobase.columns)

df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised = df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_emobase_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_emobase_normalised

Essentia Best Overall + openSMILE GeMAPS

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_gemaps), columns=df_deam_integrated_essentia_best_overall_opensmile_gemaps.columns)

df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised = df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_gemaps_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_gemaps_normalised

Essentia Best Overall + openSMILE eGeMAPS

In [ ]:
# Fit and transform the selected columns
df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised = pd.DataFrame(scaler.fit_transform(df_deam_integrated_essentia_best_overall_opensmile_egemaps), columns=df_deam_integrated_essentia_best_overall_opensmile_egemaps.columns)

df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised = df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised.drop('song_id', axis=1)
df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised.insert(0, column='song_id', value=song_ids)

df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised.to_csv(get_deam_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_egemaps_features.csv'))

df_deam_integrated_essentia_best_overall_opensmile_egemaps_normalised

## Filter out only the mean features

In [ ]:
df_essentia_best_overall_features

In [ ]:
col_names = df_essentia_best_overall_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

In [ ]:
df_essentia_best_overall_features_mean = df_essentia_best_overall_features[feature_mean_cols]
df_essentia_best_overall_features_mean

In [ ]:
df_essentia_best_overall_features_mean.to_csv(get_deam_path('processed/features/essentia_best_overall_features_mean.csv'))

Normalisation

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# Fit and transform the selected columns
df_essentia_best_overall_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_overall_features_mean), columns=df_essentia_best_overall_features_mean.columns)

df_essentia_best_overall_features_mean_normalised = df_essentia_best_overall_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_overall_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_overall_features_mean_normalised.to_csv(get_deam_path('processed/features/normalised_essentia_best_overall_features_mean.csv'))

df_essentia_best_overall_features_mean_normalised

Standardisation

In [ ]:
scaler = StandardScaler()

In [ ]:
# Fit and transform the selected columns
df_essentia_best_overall_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_overall_features_mean), columns=df_essentia_best_overall_features_mean.columns)

df_essentia_best_overall_features_mean_standardised = df_essentia_best_overall_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_overall_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_overall_features_mean_standardised.to_csv(get_deam_path('processed/features/standardised_essentia_best_overall_features_mean.csv'))

df_essentia_best_overall_features_mean_standardised

In [ ]:
df_essentia_best_arousal_features

In [ ]:
col_names = df_essentia_best_arousal_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

In [ ]:
df_essentia_best_arousal_features_mean = df_essentia_best_arousal_features[feature_mean_cols]
df_essentia_best_arousal_features_mean

In [ ]:
df_essentia_best_arousal_features_mean.to_csv(get_deam_path('processed/features/essentia_best_arousal_features_mean.csv'))

Normalisation

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# Fit and transform the selected columns
df_essentia_best_arousal_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_arousal_features_mean), columns=df_essentia_best_arousal_features_mean.columns)

df_essentia_best_arousal_features_mean_normalised = df_essentia_best_arousal_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_arousal_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_arousal_features_mean_normalised.to_csv(get_deam_path('processed/features/normalised_essentia_best_arousal_features_mean.csv'))

df_essentia_best_arousal_features_mean_normalised

Standardisation

In [ ]:
scaler = StandardScaler()

In [ ]:
# Fit and transform the selected columns
df_essentia_best_arousal_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_arousal_features_mean), columns=df_essentia_best_arousal_features_mean.columns)

df_essentia_best_arousal_features_mean_standardised = df_essentia_best_arousal_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_arousal_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_arousal_features_mean_standardised.to_csv(get_deam_path('processed/features/standardised_essentia_best_arousal_features_mean.csv'))

df_essentia_best_arousal_features_mean_standardised

In [ ]:
df_essentia_best_valence_features

In [ ]:
col_names = df_essentia_best_valence_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

In [ ]:
df_essentia_best_valence_features_mean = df_essentia_best_valence_features[feature_mean_cols]
df_essentia_best_valence_features_mean

In [ ]:
df_essentia_best_valence_features_mean.to_csv(get_deam_path('processed/features/essentia_best_valence_features_mean.csv'))

Normalisation

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# Fit and transform the selected columns
df_essentia_best_valence_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_valence_features_mean), columns=df_essentia_best_valence_features_mean.columns)

df_essentia_best_valence_features_mean_normalised = df_essentia_best_valence_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_valence_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_valence_features_mean_normalised.to_csv(get_deam_path('processed/features/normalised_essentia_best_valence_features_mean.csv'))

df_essentia_best_valence_features_mean_normalised

Standardisation

In [ ]:
scaler = StandardScaler()

In [ ]:
# Fit and transform the selected columns
df_essentia_best_valence_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_valence_features_mean), columns=df_essentia_best_valence_features_mean.columns)

df_essentia_best_valence_features_mean_standardised = df_essentia_best_valence_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_valence_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_valence_features_mean_standardised.to_csv(get_deam_path('processed/features/standardised_essentia_best_valence_features_mean.csv'))

df_essentia_best_valence_features_mean_standardised